In [17]:
# Importación librerías
import pandas as pd
import numpy as np
from pycaret.regression import *
from pycaret.time_series import *
from prophet import Prophet
from sklearn import *
from sklearn.ensemble import *
import json
from pycaret.containers.models.time_series import BaseCdsDtForecaster
from sktime.transformations.series.summarize import WindowSummarizer
from sktime.forecasting.croston import Croston
from lightgbm import LGBMRegressor
from sktime.forecasting.arima import ARIMA, AutoARIMA
from scipy.stats import uniform
from mango import Tuner
from sktime.forecasting.base import ForecastingHorizon


In [18]:
# Lectura de datos
df = pd.read_excel("reporte_diario_campaña_limpio.xlsx").drop(columns='Unnamed: 0')
festivos=pd.read_excel("prophet_festivos.xlsx")
entrenamiento_calls = pd.read_excel("validacion_diaria_calls.xlsx").drop(columns='Unnamed: 0')
entrenamiento_aht = pd.read_excel("validacion_diaria_aht.xlsx").drop(columns='Unnamed: 0')
with open('diccionario_calls.txt', 'r') as file:
    contenido = file.read()
diccionario_calls = json.loads(contenido)
for key in diccionario_calls:
    diccionario_calls[key] = eval(diccionario_calls[key])
with open('diccionario_aht.txt', 'r') as file:
    contenido = file.read()
diccionario_aht = json.loads(contenido)
for key in diccionario_aht:
    diccionario_aht[key] = eval(diccionario_aht[key])


In [19]:
# Definición error 
def wape(y,y_pred,**kwargs):
    wape = np.sum(np.abs(y - y_pred)) / np.sum(y)
    return wape

In [21]:
def nuevas_predicciones(mejor_modelo, df,linea,target,diccionario,festivos):
    datos = df[df["linea"]== linea]
    datos_prophet = datos[["fecha", target]].rename(columns={"fecha": 'ds', target: 'y'})
    state_holidays = datos[datos['state_holiday'] == 1]["fecha"].unique()
    state_holidays = pd.DataFrame({'ds': pd.to_datetime(state_holidays), 'holiday': 'state_holiday'})
    n_fecha = (pd.Timestamp('2024-12-31') - pd.to_datetime(df["fecha"]).max()).days
    nuevos_periodos = pd.DataFrame({
        "fecha": pd.date_range(start=df["fecha"].max(), periods=n_fecha + 1, freq='D')
    })

    nuevos_periodos["año"] = nuevos_periodos["fecha"].dt.year
    nuevos_periodos["mes"] = nuevos_periodos["fecha"].dt.month
    nuevos_periodos["dia"] = nuevos_periodos["fecha"].dt.day
    nuevos_periodos["dia_semana"] = nuevos_periodos["fecha"].dt.weekday


    fes_filtrado = festivos[["fecha", "state_holiday"]]
    nuevos_periodos = pd.merge(nuevos_periodos, fes_filtrado, how="left", on="fecha")
    nuevos_periodos["state_holiday"].fillna(0, inplace=True)
    nuevos_periodos.loc[nuevos_periodos['state_holiday'] == 1, 'dia_semana'] = 7

    nuevos_periodos=nuevos_periodos.set_index(pd.RangeIndex(start=datos.index[-1], stop=datos.index[-1]+len(nuevos_periodos), step=1))[["año","mes","dia","dia_semana",'state_holiday']]


    nuevos_periodos = nuevos_periodos.sort_values(by=["año", "mes", "dia", "dia_semana"]).reset_index(drop=True)
    

    if mejor_modelo == "reg":
        modelo_dic= diccionario[linea]["pycaret_reg"]
        modelo =modelo_dic["modelo"].set_params(**modelo_dic["parametros"])
        X, Y= datos[['año', 'mes', "dia", 'dia_semana', 'state_holiday']],datos[target]
        pred = modelo.fit(X, Y).predict(nuevos_periodos)
        nuevos_periodos["prediccion"]=pred
    
    elif mejor_modelo =="ts":
        modelo_dic= diccionario_calls["linea_1"]["pycaret_ts"]
        modelo =modelo_dic["modelo"].set_params(**modelo_dic["parametros"])
        X, Y= datos[['año', 'mes', "dia", 'dia_semana', 'state_holiday']],datos[target]
        if'ARIMA' in str(type(modelo_dic["modelo"])):
            fh = ForecastingHorizon(range(1, len(nuevos_periodos)+1))
            pred = modelo.fit(X = X, y = Y).predict(fh =fh,X =nuevos_periodos)
        else:
            pred = modelo.fit(X = X, y = Y).predict(X =nuevos_periodos)
        
        nuevos_periodos["prediccion"]=pred

    elif mejor_modelo== "promedio_reg_ts":
        modelo_dic_reg= diccionario[linea]["pycaret_reg"]
        modelo_reg =modelo_dic_reg["modelo"].set_params(**modelo_dic_reg["parametros"])
        X, Y= datos[['año', 'mes', "dia", 'dia_semana', 'state_holiday']],datos[target]
        pred_reg = modelo_reg.fit(X, Y).predict(nuevos_periodos)

        modelo_dic_ts= diccionario_calls["linea_1"]["pycaret_ts"]
        modelo_ts =modelo_dic_ts["modelo"].set_params(**modelo_dic_ts["parametros"])
        if'ARIMA' in str(type(modelo_dic_ts["modelo"])):
            fh = ForecastingHorizon(range(1, len(nuevos_periodos)+1))
            pred_ts = modelo_ts.fit(X = X, y = Y).predict(fh =fh,X =nuevos_periodos)
        else:
            pred_ts = modelo_ts.fit(X = X, y = Y).predict(X =nuevos_periodos)

        nuevos_periodos["prediccion"]=(pred_reg + pred_ts)/2

    elif mejor_modelo == "fb":
        prophet_params= diccionario_calls["linea_1"]["prophet"]
        model = Prophet(holidays=festivos, **prophet_params)
        model.fit(datos_prophet)
        future = model.make_future_dataframe(periods=len(nuevos_periodos), freq='D')
        forecast = model.predict(future)
        nuevos_periodos["prediccion"]=forecast["yhat"][-len(nuevos_periodos):]
    
                
    if(linea == "lines_3"): 
        nuevos_periodos.loc[nuevos_periodos['dia_semana'] == 6, 'prediccion'] = 0
        
        
    nuevos_periodos["linea"]=linea

    return nuevos_periodos
    

## Calls

In [22]:
lineas = entrenamiento_calls['linea'].unique()
wape_metrics_calls = {}
pred_calls=pd.DataFrame()

for linea in lineas:
    print(linea)
    df_linea = entrenamiento_calls[entrenamiento_calls['linea'] == linea]
    y_true = df_linea['interpolado_real_calls']
    pred_columns = ['pred_ts', 'pred_regr', 'pred_promedio', 'pred_fb', 'pred_promedio_mov']

    wape_metrics_calls[linea] = {col: round(wape(y_true, df_linea[col]),2) for col in pred_columns}
    mejor_modelo = min(wape_metrics_calls[linea], key=wape_metrics_calls[linea].get)
    predicciones_calls=nuevas_predicciones(mejor_modelo, df,linea,"interpolado_real_calls",diccionario_calls,festivos)
    pred_calls=pd.concat([pred_calls,predicciones_calls])

linea_1
linea_2
linea_3
linea_4
linea_5


In [23]:
pred_calls.to_excel("prediccion_calls.xlsx")

In [24]:
lineas = entrenamiento_aht['linea'].unique()
wape_metrics_aht = {}
pred_aht=pd.DataFrame()

for linea in lineas:
    print(linea)
    df_linea = entrenamiento_aht[entrenamiento_aht['linea'] == linea]
    y_true = df_linea['interpolado_real_aht']
    pred_columns = ['pred_ts', 'pred_regr', 'pred_promedio', 'pred_fb', 'pred_promedio_mov']

    wape_metrics_aht[linea] = {col: round(wape(y_true, df_linea[col]),2) for col in pred_columns}
    mejor_modelo = min(wape_metrics_aht[linea], key=wape_metrics_aht[linea].get)
    predicciones_aht= nuevas_predicciones(mejor_modelo,df,linea,"interpolado_real_aht",diccionario_aht,festivos)
    pred_aht=pd.concat([pred_aht,predicciones_aht])

linea_1
linea_2
linea_3
linea_4
linea_5


In [25]:
pred_aht.head()

,año,mes,dia,dia_semana,state_holiday,prediccion,linea
0,2024,4,7,6,0.0,267.4652,linea_1
1,2024,4,8,0,0.0,392.0078,linea_1
2,2024,4,9,1,0.0,387.7803,linea_1
3,2024,4,10,2,0.0,388.7924,linea_1
4,2024,4,11,3,0.0,380.5279,linea_1


In [26]:
pred_aht.to_excel("prediccion_aht.xlsx")